In [11]:
from openai import OpenAI
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from pydantic import BaseModel
import asyncio
import os
import base64
from IPython.display import Image, display
import json

In [12]:
# Load the OpenAI API key from the environment
openai_key = os.environ["OPENAI_API_KEY"]
client = OpenAI(api_key=openai_key)

In [13]:
# Simple Text Generation
response = client.responses.create(
    model="gpt-4o",
    input="tell me a joke",
)

print(response.output_text)

Why don't scientists trust atoms?

Because they make up everything!


In [14]:
# Multi-Modal Generation Text and Image 

image_Url = 'https://t4.ftcdn.net/jpg/01/71/94/77/360_F_171947753_jI2NCXjEGNscOaYhV0wvPv4vFSDnnwUM.jpg'

response = client.responses.create(
    model = "gpt-4o",
    input = [
        {"role": "user",
         "content": "what teams are playing in this image?"
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_image",
                    "image_url": image_Url
                }
            ]
        }
    ]
)

print(response.output_text)

I can't determine the teams from the image, but it looks like a group of children playing soccer in a park.


In [15]:
# Open AI Agent SDK

# Create a history tutor agent
history_tutor_agent = Agent(
    name = "History Tutor",
    handoff_description =  "Specialized agent for historical questions",
    instructions = "You provide assistance with historical queries. Explain important events and context clearly."
)

# Run the agent with a query
result = await Runner.run(history_tutor_agent, "Who was the first president of the United States?")
print(result.final_output)

The first president of the United States was George Washington. He served from 1789 to 1797 after being unanimously elected by the Electoral College. Washington is often called the "Father of His Country" for his leadership during the American Revolutionary War and his role in drafting the Constitution. His presidency set many precedents, including the formation of a cabinet and the tradition of a two-term limit.


In [17]:
# Create a math tutor agent
math_tutor_agent = Agent(
    name = "Math Tutor",
    handoff_description= "Specialist agent for math instrucitons",
    instructions = "You provide help with math problems. Explain your reasoning at each step and include examples."
)

# Run the math agent
result = await Runner.run(math_tutor_agent, "what is the power of x**2 - 1")
print(result.final_output)


When you refer to the "power of" an expression like \(x^2 - 1\), it usually concerns the degree of the polynomial.

### Degree of a Polynomial
The degree of a polynomial is the highest power of the variable \(x\) in the expression when it is expanded completely.

### Example

For the polynomial \(x^2 - 1\):

1. **Identify the Terms**: The polynomial consists of two terms: \(x^2\) and \(-1\).

2. **Degrees of Each Term**:
   - The term \(x^2\) has a degree of 2.
   - The constant term \(-1\) has a degree of 0 (since constants can be thought of as multiplied by \(x^0\)).

3. **Highest Degree Term**: Among \(x^2\) and \(-1\), the term with the highest degree is \(x^2\), which has a degree of 2.

Thus, the degree of the polynomial \(x^2 - 1\) is 2. Therefore, when you are asked for the "power of" this expression, it typically means the same as the degree of the polynomial, which is 2.


In [ ]:
# Recreate specialized agents

history_tutor_agent = Agent(
    name = "History Tutor",
    handoff_description="Specialized agent for historical questions",
    instructions= "You provide assistance with historical queries. Explain important events and context clearly."
)

math_tutor_agent = Agent(
    name = "Math Tutor",
    handoff_description= "Specialist agent for math instructions",
    instructions = "You provide help with math problems. Explain your reasoning at each step and include examples."
)

triage_agent = Agent(
    name = "Triage Agent",
    instructions="Handoff to the appropriate agent based on the nature of the request",
    handoffs=[history_tutor_agent, math_tutor_agent]
)

result = await Runner.run(triage_agent, "What is derivative of x^2 - 1")
print(result.final_output)

result = await Runner.run(triage_agent, "Who was the first president of the united states?")
print(result.final_output)